<a href="https://colab.research.google.com/github/ucfilho/AI/blob/main/AI_2023/Stochastic/class_05_PSO_HEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np

In [ ]:
# Target: temperatures initial and target
Target = {'H1':[327,40],'H2':[220,160],'H3':[220,60],
          'H4':[160,45],
          'C1':[100,300],'C2':[35,164],'C3':[85,138],
          'C4':[60,170],'C5':[140,300]}

Util = {'cold':[15,30],'hot':[350,250]}

# Temperatures (hot streams first, cold after and starting with H/C for hot and cold) otherwise need to adapt the code
Prop = {'H1':[100, 0.5 ],'H2':[160, 0.4 ],'H3':[60, 0.14],
        'H4':[400,0.30 ],
        'C1':[100,0.35 ],'C2':[70, 0.70 ],'C3':[350, 0.50],
        'C4':[60,0.14 ],'C5':[200, 0.60],'cold':[4.186,0.5],'hot':[4.186,0.5]} # CP e H
n_hot = 4

# w input of swarm intelligence size = n_exchages*num_hot
n_trocas = 3

In [ ]:
def Cost_find(w,Target = Target, Prop = Prop, Util= Util, n_trocas = n_trocas, ans = False ):
  w = list(w)
  Gasta_cold = [] # [Area,flow,kwy] for each stream
  Gasta_hot = []  # [Area,flow,kwy] for each stream
  U_q =[]
  names = list(Target.keys())
  n_hot, n_cold, stream= 0, 0, ['No']
  # constroi temperaturas
  Temp ={}
  for word in names:
    Temp[word]=[Target[word][0]]

  for word in names:
    if 'H' in word:
      n_hot += 1
    elif 'C' in word:
      n_cold += 1
      stream.append(word)
  
  N = len(stream)
  r = 1/N
  trocas =[]
  Areas = []
  n = n_trocas*n_hot
  
  x = w[0:n]
  tx = w[n:(2*n)]

  cont = 0
  for xi in x:
    total = 0
    cont += 1
    for k in range(N):
      total += r
      if xi <= total:
        trocas.append(stream[k])
        break 
  n1, n2, matrix_trocas, xt_hot = 0, 0, [],[]

  while(len(trocas) < n):
    trocas.append('No')
  
  num = int(len(trocas)/n_hot)
  # print('num',num, trocas)

  for idx in range(n_hot):
    tr, dr =  [], []
    n2 = num*(1+idx)
    tr.append(trocas[n1:n2])
    dr.append(tx[n1:n2])
    matrix_trocas.append(tr[0])
    xt_hot.append(dr[0])
    n1 = n2
  
  idx = 0
  # update hot temperature
  for j in range(n_trocas):
    for k in range(n_hot):
      x = Temp[names[k]][idx]
      # print('matrix_trocas, k,j',matrix_trocas, k,j)
      cold = matrix_trocas[k][j]
      hot = names[k]
      if cold == 'No':
        Temp[names[k]].append(x)
      else:
        # x, y = hot and cold temperatures
        dx = Target[hot][0]-Target[hot][1]
        x0 = x* 1.0
        y0 = Temp[cold][-1]
        x = x - dx * xt_hot[k][j]
        CP_c = Prop[cold][0]
        CP_h = Prop[hot][0]
        Q = CP_c* dx * xt_hot[k][j]
        y = CP_h*dx * xt_hot[k][j]/CP_c + Temp[cold][-1]
        d1 = x0- y
        d2 = x - y0
        U = 1.0 / ( 1.0/Prop[cold][1]+ 1.0/Prop[hot][1])
        if d2 != 0 and d1*d2 > 0:
          check = 'ok'
        else:
          check = 'no'
        if x0 > y and x > y0 and check == 'ok': # so troca calor se a temperatura fria final for menor q a temperatura quente final
          Temp[cold].append(y)
          Temp[names[k]].append(x)
          LMDT = (d1 - d2) / np.log(d1/d2)
          A = Q /(U*LMDT) 
          Areas.append(A)
          U_q.append(U)
        else:
          y = Temp[cold][-1]
          x = Temp[names[k]][-1]
          Temp[cold].append(y)
          Temp[names[k]].append(x)       
    idx += 1

  cp_hot = Prop['hot'][0] # cp
  dt = Util['cold'][1]-Util['cold'][0]
  cp_cold = Prop['cold'][0]*dt 

  for word in names:
    x0 = Temp[word][-1]
    x = Target[word][1]
    cp = Prop[word][0]
    Q = cp*abs(x-x0)
    
    if x0 < x : # cooling activate
      flow = Q/cp_cold
      U = 1.0 / ( 1.0/Prop[word][1]+ 1.0/Prop['cold'][1])
      y0 = Util['cold'][0]
      y = Util['cold'][1]
      d1 = x0- y
      d2 = x - y0   
      if d2 != 0 and d1*d2 > 0:
        check = 'ok'
      else:
        check = 'no'
      

      # so troca calor se a temperatura fria final for menor q a temperatura quente final
      if x0 > y and x > y0 and check == 'ok': 
        #print('d1,d2',d1,d2)
        LMDT = (d1 - d2) / np.log(d1/d2)
        A = Q /(U*LMDT) # convert MW to kW
        kW = Q
        Gasta_cold.append([A,flow,kW])   
    else:
      flow = Q / cp_hot 
      U = 1.0 / ( 1.0/Prop[word][1]+ 1.0/Prop['cold'][1])
      y0 = Util['hot'][0]
      y = Util['hot'][1]
      d1 = y-x0 
      d2 = y0  -x   
      if d2 != 0 and d1*d2 > 0:
        check = 'ok'
      else:
        check = 'no'
      if x0 < y and x < y0 and check == 'ok': # so troca calor se a temperatura fria final for menor q a temperatura quente final
        #print('d1,d2',d1,d2)
        LMDT = (d1 - d2) / np.log(d1/d2)
        A = Q /(U*LMDT)
        kW= Q
        Gasta_hot.append([A,flow,kW])   

  Cost = 0
  #a, n, h, c = 4333,0.6, 150.53, 53.064 # L.V. Pavao et al. / Energy 142 (2018) 356e372
  a1,a2, h, c = 2000, 70,  60, 6 # Chemical Engineering Science 158 (2017) 96–107- Pavao et al

  for A in Areas:
    Cost += a1+a2*A #a*A**n
  for x in Gasta_hot:
    Cost += a1+a2*A #a*A**n
    Cost += h*x[2] # x[2]: steam 

  for x in Gasta_cold:
    Cost += a1+a2*A # a*x[0]**n
    Cost += c*x[2] # x[2]: cold water kW
  if np.isnan(Cost)==True:
    Cost = 1e99
  if ans == False:
    return Cost
  else:
    return Cost, matrix_trocas, Temp

In [ ]:
w = [0.3]*4+[0.8]*4+[0.5]*4+[1]*12
print("%.4e"%Cost_find(w))

4.9949e+06


In [ ]:
fitness_function = Cost_find

In [ ]:
def fun(x):
  n = x.shape[0]
  ans = np.zeros(n)
  for i in range(n):
    ans[i] = fitness_function(x[i,:])
  return ans

In [ ]:
def pop_rnd(population,dimension,Max,Min):
  x = np.zeros((population,dimension))
  for i in range(population):
    for j in range(dimension):
      r = np.random.rand()
      x[i,j] = Min[j] + r*(Max[j]-Min[j])
  return x

In [ ]:
def best(xp,xg,fp,fg,fitness,x):
  # gbest, pbest = solution vector with best function 
  #                (global and particle gen)
  # fitness = values of function all particles
  # best = values of function best all iterations
  n = x.shape[0]

  for i in range(n):
    if fp[i] >= fitness[i]:
      xp[i,:] = x[i,:]*1.0
      fp[i] = fitness[i]*1.0
    if fg >= fitness[i]:
      xg = xp[i,:]*1.0
      fg = fitness[i]* 1.0
  return xp,fp,xg,fg

In [ ]:
def melhora(x,fit,xnew):
  n = x.shape[0]
  fnew = fun(xnew)
  for i in range(n):
    if fit[i] >= fnew[i]:
      x[i,:] = xnew[i,:]
      fit[i] = fnew[i]
  
  return x,fit

In [ ]:
def PSO(D,pop,gen,Max,Min):
  fp = np.ones(pop)*1e99
  v = np.zeros((pop,D))
  vnew = np.zeros((pop,D))
  xnew = np.zeros((pop,D))
  fg = 1e99
  xg = np.ones((1,D))
  xp = np.ones((pop,D))
  x = pop_rnd(pop,D,Max,Min)
  w, c1, c2 =0.8,  0.1, 0.1
  fit = fun(x)
  #print('fit', fit)
  #print(x)
  for k in range(gen):
    xp,fp,xg,fg = best(xp,xg,fp,fg,fit,x)
    for i in range(pop):
      for j in range(D):
        r1 = np.random.rand()
        r2 = np.random.rand()
        vnew[i,j] = w*v[i,j]- c1*r1*(xp[i,j]-x[i,j])+ c2*r2*(xg[j]-x[i,j])
        v[i,j] = vnew[i,j]* 1.0
        xnew[i,j] = x[i,j] - vnew[i,j]
        if xnew[i,j] > Max[j] or xnew[i,j] < Min[j]:
          xnew[i,j] = Min[j] + r1*(Max[j]-Min[j])

    x,fit = melhora(x,fit,xnew)  
  
  return xg,fg

In [ ]:
D = n_hot*n_trocas* 2
pop = 300
gen = 300
NPAR=1 #Numero de fontes de comida
ITE=1 #ITERACOES (maxCycle)
PAR=n_hot*n_trocas* 2 #NUM DE PARAMETROS A SER OTIMIZADOS
print(40*'=',PAR,40*'=')
MAX=[] # MAXIMO DE CADA PARAMETRO
MIN=[] # MINIMO DE CADA PARAMETRO
for k in range(PAR):
  MAX.append(1)
  MIN.append(0)
xg,fg = PSO(D,pop,gen,MAX,MIN)
print(xg)
print(fg)

======================================== 24 ========================================


<ipython-input-3-1e6592f7a7f0>:85: RuntimeWarning: invalid value encountered in double_scalars
  LMDT = (d1 - d2) / np.log(d1/d2)


[0.6843474  0.11257028 0.94378863 0.23561732 0.14829883 0.54148486
 0.22735896 0.4391048  0.97083969 0.98556452 0.65747813 0.41613978
 0.75163701 0.99288088 0.15036422 0.89268135 0.98943804 0.96307831
 0.25958993 0.88416972 0.96668884 0.08814145 0.40346888 0.24530397]
2600793.653458099


In [ ]:
for resp in Cost_find(xg, ans=True):
  print(resp)

2600793.653458099
[['C4', 'No', 'C5'], ['C1', 'No', 'C3'], ['C1', 'C2', 'C5'], ['C5', 'C3', 'C2']]
{'H1': [327, 327, 327, 283.84546964284846], 'H2': [220, 166.43911903376062, 166.43911903376062, 166.43911903376062], 'H3': [220, 220, 78.53284430299172, 78.53284430299172], 'H4': [160, 160, 113.60107901740571, 113.60107901740571], 'C1': [100, 185.697409545983, 185.697409545983], 'C2': [35, 156.25756202600712, 156.25756202600712], 'C3': [85, 138.02733826582204, 138.02733826582204], 'C4': [60, 60], 'C5': [140, 140, 161.57726517857577, 161.57726517857577]}
